In [6]:
pip install elsapy

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install xlwt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 2.2 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [23]:
from elsapy.elsclient import ElsClient
from elsapy.elsprofile import ElsAuthor, ElsAffil
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch
import json
import re
import csv
import xlwt



def read_json(input):
    author = []
    all_author = input["coredata"]['dc:creator']

    for v in all_author:
        try:
            author.append(v['$'])
        except TypeError as e:
            print(e)
            print(v)
            return [all_author['$']]
        return author

def get_pii(url): #get pii (the id of articles on ScienceDirect) with doi
    value = re.search("pii/S.*",url).group()
    
    return value[4:]

def get_doi(url): #git full doi with initalized doi  
    value = re.search("org/10.*",url).group()
    return value[4:]


if __name__ == "__main__":
    ## Load configuration
    con_file = open("../data/config.json")
    config = json.load(con_file)
    con_file.close()

    ## Initialize client
    client = ElsClient(config['apikey'])

    all_datapaper_doi = []
    all_originalpaper_doi = []
    with open("../data/Rawdata.csv", encoding="utf-8") as f:
        for row in csv.reader(f, skipinitialspace=False):
            doiurl = "https://doi.org/" + row[0]
            all_datapaper_doi.append(doiurl) #get dois from RawData
            piiurl = "https://www.sciencedirect.com//science/article/pii/" + row[1]
            all_originalpaper_doi.append(piiurl)
        f.close()

    file = open("../output/data_paper_author_output.csv", 'a+', encoding= "utf-8-sig", newline='')
    csv_writer = csv.writer(file)
    

    for item in all_originalpaper_doi:
        rank = 0
        print ("12")
        if item == "":
            print("222") #the print out here is used for monitoring the progress and errors when the code is running
            break
        pii = get_pii(item)
        pii_doc = FullDoc(sd_pii=pii)

        if pii_doc.read(client):
            all_data = pii_doc.data
            document_doi = pii_doc.uri
            pii_doc.write()
            authors = read_json(all_data) #get the author information from the json data returned by Scopous API

            for person in authors:
                rank = rank + 1
                csv_writer.writerow([item, document_doi, person, rank])
        else:
            print ("Read document failed.")
            csv_writer.writerow([item,"failed","failed"])

    for item in all_datapaper_doi:
        rank = 0
        if item is None:
            break
        k = get_doi(item)
        pii_doc = FullDoc(doi= k)

        if pii_doc.read(client):
            all_data = pii_doc.data
            document_doi = pii_doc.uri
            pii_doc.write()
            authors = read_json(all_data) ##get the author information from the json data returned by Scopous API
            for person in authors:
                rank = rank + 1

                csv_writer.writerow([item, document_doi, person, rank])
        else:
            print ("Read document failed.")
            csv_writer.writerow([item,"failed","failed"])

    file.close()

12
12
12
12


KeyboardInterrupt: 